In [1]:
from line_solver import *
import numpy as np
GlobalConstants.set_verbose(VerboseLevel.SILENT)

# Clear variables (equivalent to MATLAB's clear solver AvgTable)
solver = None
AvgTable = None

In [ ]:
# MATLAB: model = LayeredNetwork('myLayeredModel');
model = LayeredNetwork('myLayeredModel')

# definition of processors, tasks and entries
# MATLAB: P1 = Processor(model, 'P1', 1, SchedStrategy.PS);
# MATLAB: T1 = Task(model, 'T1', 100, SchedStrategy.REF).on(P1);
# MATLAB: E1 = Entry(model, 'E1').on(T1);
P1 = Processor(model, 'P1', 1, SchedStrategy.PS)
T1 = Task(model, 'T1', 100, SchedStrategy.REF).on(P1)
E1 = Entry(model, 'E1').on(T1)

# MATLAB: P2 = Processor(model, 'P2', 1, SchedStrategy.PS);
# MATLAB: T2 = Task(model, 'T2', 1, SchedStrategy.INF).on(P2);
# MATLAB: E2 = Entry(model, 'E2').on(T2);
# MATLAB: E3 = Entry(model, 'E3').on(T2);
P2 = Processor(model, 'P2', 1, SchedStrategy.PS)
T2 = Task(model, 'T2', 1, SchedStrategy.INF).on(P2)
E2 = Entry(model, 'E2').on(T2)
E3 = Entry(model, 'E3').on(T2)

# definition of activities
# MATLAB: T1.set_think_time(Erlang.fit_mean_and_order(10,1));
T1.set_think_time(Erlang.fit_mean_and_order(10, 1))

# MATLAB: A1 = Activity(model, 'A1', Exp(1)).on(T1).boundTo(E1).synchCall(E2).synchCall(E3,1);
A1 = Activity(model, 'A1', Exp(1)).on(T1).bound_to(E1).synch_call(E2).synch_call(E3, 1)

# MATLAB: A20 = Activity(model, 'A20', Exp(1)).on(T2).boundTo(E2);
# MATLAB: A21 = Activity(model, 'A21', Exp(1)).on(T2);
# MATLAB: A22 = Activity(model, 'A22', Exp(1)).on(T2).repliesTo(E2);
# MATLAB: T2.add_precedence(ActivityPrecedence.Serial(A20, A21, A22));
A20 = Activity(model, 'A20', Exp(1)).on(T2).bound_to(E2)
A21 = Activity(model, 'A21', Exp(1)).on(T2)
A22 = Activity(model, 'A22', Exp(1)).on(T2).replies_to(E2)
T2.add_precedence(ActivityPrecedence.Serial(A20, A21))
T2.add_precedence(ActivityPrecedence.Serial(A21, A22))

# MATLAB: A3 = Activity(model, 'A3', Exp(1)).on(T2).boundTo(E3).repliesTo(E3);
A3 = Activity(model, 'A3', Exp(1)).on(T2).bound_to(E3).replies_to(E3)

In [3]:
# instantiate solvers
solvers = []
avg_tables = []

# LQNS solver
options = LQNS.default_options()
options.keep = True
options.verbose = 0
if LQNS.isAvailable():
    solver_lqns = LQNS(model, options)
    avg_table_lqns = solver_lqns.avg_table()

   Node   NodeType     QLen  Util     RespT  ResidT  ArvR  Tput
0    P1  Processor      NaN  0.25       NaN     0.0   0.0   NaN
1    P2  Processor      NaN  1.00       NaN     0.0   0.0   NaN
2    T1    RefTask  97.5000  0.25       NaN     0.0   0.0  0.25
3    T2       Task  97.1681  1.00       NaN     0.0   0.0  0.50
4    E1      Entry  97.5000  0.25  390.0000     0.0   0.0  0.25
5    E2      Entry  72.8761  0.75  291.5040     0.0   0.0  0.25
6    E3      Entry  24.2920  0.25   97.1682     0.0   0.0  0.25
7    A1   Activity  97.5000  0.25  390.0000     0.0   0.0  0.25
8   A20   Activity  24.2920  0.25   97.1682     0.0   0.0  0.25
9   A21   Activity  24.2920  0.25   97.1682     0.0   0.0  0.25
10  A22   Activity  24.2920  0.25   97.1682     0.0   0.0  0.25
11   A3   Activity  24.2920  0.25   97.1682     0.0   0.0  0.25


In [4]:
# LN solver with NC
lnoptions = LN.default_options()
lnoptions.verbose = 0
solveroptions = NC.default_options()
solveroptions.verbose = 0
solver_nc = LN(model, lambda l: NC(l, solveroptions), lnoptions)
avg_table_nc = solver_nc.avg_table()

LN initialization completed. Starting iteration on ensemble models.
Iter 1.Analyze time: 0.038810s. Update time: 0.003394s. Runtime: 0.039157s.
Iter 2.Analyze time: 0.002368s. Update time: 0.001933s. Runtime: 0.046235s.
Iter 3.Analyze time: 0.002085s. Update time: 0.002433s. Runtime: 0.050512s.
Iter 4.Analyze time: 0.002923s. Update time: 0.002368s. Runtime: 0.056037s.
Iter 5.Analyze time: 0.001459s. Update time: 0.002526s. Runtime: 0.060042s.
Iter 6.Analyze time: 0.001773s. Update time: 0.001918s. Runtime: 0.064504s.
Iter 7.Analyze time: 0.001612s. Update time: 0.002017s. Runtime: 0.068272s.
Iter 8.Analyze time: 0.001911s. Update time: 0.003765s. Runtime: 0.072507s.
Iter 9.Analyze time: 0.002732s. Update time: 0.001786s. Runtime: 0.079853s.
Iter 10.Analyze time: 0.001150s. Update time: 0.001429s. Runtime: 0.083850s.
Iter 11.Analyze time: 0.001054s. Update time: 0.001669s. Runtime: 0.086685s.
Iter 12.Analyze time: 0.000993s. Update time: 0.001623s. Runtime: 0.089463s.
Iter 13.Analyze t

In [5]:
print(avg_table_nc)

   Node   NodeType     QLen    Util    RespT   ResidT  ArvR    Tput
0    P1  Processor      NaN  1.0000      NaN      NaN   NaN     NaN
1    P2  Processor      NaN  0.3220      NaN      NaN   NaN     NaN
2    T1       Task  86.9737  1.0000      NaN  87.6312   NaN  1.0000
3    T2       Task   0.0000  0.3220      NaN   2.0000   NaN  0.1610
4    E1      Entry  86.9737     NaN  86.9719      NaN   NaN  1.0000
5    E2      Entry   0.0000     NaN   0.0000      NaN   NaN  0.0805
6    E3      Entry   0.0000     NaN   0.0000      NaN   NaN  0.0805
7    A1   Activity  89.2112  1.0000  89.2094  87.6312   NaN  1.0000
8   A20   Activity   0.0805  0.0805   1.0000   0.5000   NaN  0.0805
9   A21   Activity   0.0805  0.0805   1.0000   0.5000   NaN  0.0805
10  A22   Activity   0.0805  0.0805   1.0000   0.5000   NaN  0.0805
11   A3   Activity   0.0805  0.0805   1.0000   0.5000   NaN  0.0805
